# Q5：QKV的作用是什么？？？

> 答：
> Q：即query，个人理解为用户的输入，KV是内置的矩阵，注意力机制根据Q和K的相似度去得到对应的V
> 
> 输入的X经过Embedding层，维度变成了 batch_size * num_steps/query_nums * num_hiddens
> 
> 如果假设 KV 的维度都为 batch_size * KV个数 * num_hiddens，并且使用点积注意力，则有
> 
> 将 X 与 K 使用批量矩阵乘法，得到的维度为 batch_size * query_nums * KV_nums
> 
> 我们可以把得到的矩阵称作评分矩阵，对于批量中的每一行，我们可以理解为一个词元在V上的分数
> 
> 然后输入masked_softmax, 得到对应的权重矩阵,
> 
> 接下来，和V矩阵执行一个批量矩阵乘法，得到最后的结果V
> 


# 多头注意力

个人的理解来使，多头注意力就像是把长文本分解成小文本，然后查找小文本间的关系，具体的做法可以体现在矩阵乘法中

如果是直接的矩阵乘法，那么Query会捕捉到所有Key的信息

但是如果是多头注意力，Query和Key，Value都被分成了部分区域，来获取对应的区域的关系

In [180]:
import math
import torch
import torch.nn as nn

In [181]:
# start: copy from [https://zh-v2.d2l.ai/]
def masked_softmax(x, valid_lens, eps=1e-8):
    # X为评分矩阵
    # shape X: batch * num_steps * num_steps
    if valid_lens.shape[0] == 0:
        return nn.functional.softmax(x, dim=1)
    shape = x.shape
    if valid_lens.dim() == 2:
        valid_lens = valid_lens.reshape(-1)
    mask_bool = torch.arange(shape[1], device=x.device).unsqueeze(0) >= valid_lens.unsqueeze(1)
    mask_bool = mask_bool.unsqueeze(1).expand(-1, shape[1], -1)
    x = x.masked_fill(mask_bool, eps)
    return nn.functional.softmax(x, dim=1)

masked_softmax(torch.rand(2, 4, 4), torch.tensor([2, 3]))

tensor([[[0.3189, 0.2730, 0.2500, 0.2500],
         [0.1722, 0.1620, 0.2500, 0.2500],
         [0.1836, 0.2661, 0.2500, 0.2500],
         [0.3253, 0.2989, 0.2500, 0.2500]],

        [[0.2611, 0.2925, 0.2280, 0.2500],
         [0.2152, 0.1778, 0.2162, 0.2500],
         [0.2417, 0.1472, 0.3619, 0.2500],
         [0.2820, 0.3824, 0.1939, 0.2500]]])

In [182]:
class DotAttention(nn.Module):
    def __init__(self, **kwargs):
        super(DotAttention, self).__init__(**kwargs)
        
    def forward(self, q, k, v, valid_lens=None):
        d = q.shape[-1]
        score_matrix = torch.bmm(q, k.transpose(1, 2)) / math.sqrt(d)
        attention_weights = masked_softmax(score_matrix, valid_lens)
        print(f"attention weights shape: {attention_weights.shape}, v shape: {v.shape}")
        return torch.bmm(attention_weights, v)

In [183]:
dot_attention = DotAttention()
dot_attention.eval()
q = torch.normal(0, 1, (2, 256, 256))
k = torch.normal(0, 1, (2, 256, 256))
v = torch.normal(0, 1, (2, 256, 256))
dot_attention(q, k, v, valid_lens=torch.tensor([123, 100]))

attention weights shape: torch.Size([2, 256, 256]), v shape: torch.Size([2, 256, 256])


tensor([[[-0.1233, -0.0260, -0.0146,  ..., -0.1036, -0.1355,  0.0510],
         [-0.0619, -0.0040,  0.0063,  ..., -0.2192, -0.1016, -0.1287],
         [-0.1955, -0.1010, -0.0374,  ..., -0.1508, -0.1476, -0.0293],
         ...,
         [-0.1709, -0.0853, -0.0533,  ..., -0.2309, -0.0764, -0.0142],
         [-0.1737, -0.0315,  0.0538,  ..., -0.1339, -0.1313, -0.0381],
         [-0.1507, -0.0925,  0.0256,  ..., -0.2305, -0.1589, -0.0680]],

        [[ 0.0412, -0.0049,  0.0784,  ..., -0.0690,  0.0115,  0.1024],
         [ 0.0839,  0.0811,  0.1457,  ..., -0.1552, -0.0115,  0.0325],
         [ 0.0458,  0.0422,  0.0523,  ..., -0.1096, -0.0748, -0.0198],
         ...,
         [ 0.1051, -0.0105,  0.1152,  ..., -0.1466, -0.0136,  0.1193],
         [ 0.1182, -0.0369,  0.0908,  ..., -0.1837, -0.0329, -0.0035],
         [-0.0019, -0.0338,  0.1062,  ..., -0.2575, -0.0454,  0.0162]]])

In [184]:
def transpose_qkv(X, num_heads):
    # X: batch * num_steps * num_hiddens
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)
    # X: batch * num_steps * num_head * (num_hiddens / num_head)
    X = X.permute(0, 2, 1, 3)
    # X: batch * num_head * num_steps * (num_hiddens / num_head)
    
    # final: (batch*num_head) * num_steps * (num_hiddens / num_head)
    return X.reshape(-1, X.shape[2], X.shape[3])


def transpose_output(X, num_heads):
    # X: (batch*num_head) * num_steps * (num_hiddens / num_head)
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1],-1)

In [185]:
class SimpleMultiHeadAttention(nn.Module):
    def __init__(self, q_size, k_size, v_size, num_hiddens, num_head,  **kwargs):
        super(SimpleMultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_head
        self.attention = DotAttention()
        self.W_q = nn.Linear(q_size, num_hiddens)
        self.W_k = nn.Linear(k_size, num_hiddens)
        self.W_v = nn.Linear(v_size, num_hiddens)
        self.W_out = nn.Linear(num_hiddens, num_hiddens)
        
        
    def forward(self, query, key, value, valid_lens=None):
        queries = transpose_qkv(self.W_q(query), self.num_heads)
        keys = transpose_qkv(self.W_k(key), self.num_heads)
        values = transpose_qkv(self.W_v(value), self.num_heads)
        
        if valid_lens is not None:
            # 重复head次
            valid_lens = torch.repeat_interleave(
                valid_lens, repeats=self.num_heads, dim=0
            )
        
        print(f"Query shape: {queries.shape}, Keys shape: {keys.shape}, Values shape: {values.shape}")
        output = self.attention(queries, keys, values, valid_lens)
        output = transpose_output(output, num_heads=self.num_heads)
        return self.W_out(output)
    
    
multi = SimpleMultiHeadAttention(256,256,256,256,num_head=4)
multi.eval()
multi(q, k, v, valid_lens=torch.tensor([123, 100]))
# end: copy from [https://zh-v2.d2l.ai/]

Query shape: torch.Size([8, 256, 64]), Keys shape: torch.Size([8, 256, 64]), Values shape: torch.Size([8, 256, 64])
attention weights shape: torch.Size([8, 256, 256]), v shape: torch.Size([8, 256, 64])


tensor([[[-0.0557,  0.0119, -0.0074,  ...,  0.0495, -0.0769,  0.0474],
         [-0.0561,  0.0053, -0.0051,  ...,  0.0487, -0.0715,  0.0486],
         [-0.0577,  0.0137, -0.0096,  ...,  0.0481, -0.0862,  0.0477],
         ...,
         [-0.0588,  0.0072, -0.0023,  ...,  0.0532, -0.0747,  0.0482],
         [-0.0565,  0.0101, -0.0178,  ...,  0.0457, -0.0779,  0.0447],
         [-0.0543,  0.0072,  0.0015,  ...,  0.0427, -0.0763,  0.0485]],

        [[-0.0310,  0.0131,  0.0041,  ...,  0.0615, -0.0456,  0.0320],
         [-0.0306,  0.0121, -0.0087,  ...,  0.0585, -0.0530,  0.0337],
         [-0.0377,  0.0181, -0.0061,  ...,  0.0681, -0.0515,  0.0344],
         ...,
         [-0.0294,  0.0116, -0.0047,  ...,  0.0554, -0.0594,  0.0408],
         [-0.0305,  0.0184, -0.0038,  ...,  0.0652, -0.0532,  0.0406],
         [-0.0268,  0.0127,  0.0036,  ...,  0.0558, -0.0582,  0.0430]]],
       grad_fn=<ViewBackward0>)

# GQA 分组多头注意力

## 好处：
1. 据我所知是结合了多头注意力和多查询注意力MQA的折中选择，使用MQA的话无法捕捉到局部信息，使用MHA的话可能速度会下降，因此进行了折中，同时，GQA的两个极端其实就是MQA和MHA
2. 我个人感觉想是对局部信息的又一个扩展吧，就是可能会有不同head的queries会用的是同一个k来捕捉关系，全部使用MHA的话反而会丢失这种关系，果然还得是中庸之道啊

In [186]:
# start: copy from [https://zh-v2.d2l.ai/]
def transpose_q_group(X, num_heads, num_groups):
    # X: batch * num_steps * num_hiddens
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)
    # X: batch * num_steps * num_head * (num_hiddens / num_head)
    X = X.permute(0, 2, 1, 3)
    # X: batch * num_head * num_steps * (num_hiddens / num_head)
    # : batch * num_group * group_size * num_steps * num_hiddens/num_head
    return X.reshape(X.shape[0], num_groups, num_heads//num_groups, X.shape[2], X.shape[3])

def transpose_kv_group(X, num_groups):
    X = X.reshape(X.shape[0], X.shape[1], num_groups, -1)
    # batch * group * num_steps * num_hiddens/num_steps
    return X.permute(0, 2, 1, 3)
# end: copy from [https://zh-v2.d2l.ai/]

In [187]:
# start: copy from [GPT]
class GroupedQAAttention(nn.Module):
    def __init__(self, q_size, k_size, v_size, num_hiddens, num_heads, num_groups, **kwargs):
        super(GroupedQAAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.num_groups = num_groups
        self.group_size = num_heads // num_groups
        self.attention = DotAttention()
        self.W_q = nn.Linear(q_size, num_hiddens)
        self.W_k = nn.Linear(k_size, num_hiddens // self.group_size)
        self.W_v = nn.Linear(v_size, num_hiddens // self.group_size)
        self.W_out = nn.Linear(num_hiddens, num_hiddens)

    def forward(self, query, key, value, valid_lens=None):
        # query: batch num_group group_size num_steps num_hiddens/num_steps
        # kv：batch*num_group * num_steps * num_hiddens/num_group
        # print(f"0 => Query shape: {query.shape}, Keys shape: {key.shape}, Values shape: {value.shape}")
        queries = transpose_q_group(self.W_q(query), self.num_heads, self.num_groups)
        keys = transpose_kv_group(self.W_k(key), self.num_groups)
        values = transpose_kv_group(self.W_v(value), self.num_groups)
        # print(f"1 => Queries shape: {queries.shape}")
        output_groups = []
        for i in range(self.num_groups):
            group_queries = queries[:, i, :, :]  # 提取当前组的 queries
            group_keys = keys[:, i, :, :]
            group_values = values[:, i, :, :]
            # print(f"2 => Group queries shape: {group_queries.shape}, group_keys shape: {group_keys.shape}, group_keys shape: {group_values.shape}")
            for j in range(self.group_size):
                # if valid_lens is not None:
                #     valid_lens_group = torch.repeat_interleave(valid_lens, repeats=group_queries.shape[-1], dim=0)
                # else:
                #     valid_lens_group = None
                # 计算注意力输出
                group_output = self.attention(group_queries[:,j, :], group_keys, group_values, valid_lens)
                # print(f"3 => Group_output shape: {group_output.shape}")
                output_groups.append(group_output)
        
        # print(f"4 => Group outputs shape: {output_groups[0].shape}")
        # 拼接所有组的输出
        output = torch.cat(output_groups, dim=2)
        # 维度变化：
        # 拼接后的 `output` 形状: `(batch, num_steps, num_hiddens)`
        return self.W_out(output)
    
# end: copy from [GPT]

In [188]:
group = GroupedQAAttention(256,256,256,256,num_heads=8, num_groups=4)
group.eval()
# qkv batch * num_steps * num_hiddens
group(q, k, v, valid_lens=torch.tensor([123, 100]))

0 => Query shape: torch.Size([2, 256, 256]), Keys shape: torch.Size([2, 256, 256]), Values shape: torch.Size([2, 256, 256])
1 => Queries shape: torch.Size([2, 4, 2, 256, 32])
2 => Group queries shape: torch.Size([2, 2, 256, 32]), group_keys shape: torch.Size([2, 256, 32]), group_keys shape: torch.Size([2, 256, 32])
attention weights shape: torch.Size([2, 256, 256]), v shape: torch.Size([2, 256, 32])
3 => Group_output shape: torch.Size([2, 256, 32])
attention weights shape: torch.Size([2, 256, 256]), v shape: torch.Size([2, 256, 32])
3 => Group_output shape: torch.Size([2, 256, 32])
2 => Group queries shape: torch.Size([2, 2, 256, 32]), group_keys shape: torch.Size([2, 256, 32]), group_keys shape: torch.Size([2, 256, 32])
attention weights shape: torch.Size([2, 256, 256]), v shape: torch.Size([2, 256, 32])
3 => Group_output shape: torch.Size([2, 256, 32])
attention weights shape: torch.Size([2, 256, 256]), v shape: torch.Size([2, 256, 32])
3 => Group_output shape: torch.Size([2, 256, 32

tensor([[[ 0.0325, -0.0903, -0.0593,  ...,  0.0716, -0.0988, -0.0093],
         [ 0.0369, -0.0721, -0.0563,  ...,  0.0715, -0.0904, -0.0117],
         [ 0.0296, -0.0947, -0.0687,  ...,  0.0699, -0.0987, -0.0059],
         ...,
         [ 0.0382, -0.0914, -0.0571,  ...,  0.0708, -0.0933, -0.0089],
         [ 0.0300, -0.0783, -0.0697,  ...,  0.0816, -0.0940, -0.0024],
         [ 0.0334, -0.0864, -0.0539,  ...,  0.0630, -0.0980, -0.0129]],

        [[ 0.0315, -0.0738, -0.0307,  ...,  0.0611, -0.1061, -0.0052],
         [ 0.0110, -0.0816, -0.0333,  ...,  0.0671, -0.0957, -0.0051],
         [ 0.0184, -0.0789, -0.0242,  ...,  0.0615, -0.1061, -0.0038],
         ...,
         [ 0.0288, -0.0727, -0.0297,  ...,  0.0625, -0.1097, -0.0085],
         [ 0.0282, -0.0793, -0.0236,  ...,  0.0586, -0.1111,  0.0068],
         [ 0.0293, -0.0836, -0.0368,  ...,  0.0541, -0.1077, -0.0106]]],
       grad_fn=<ViewBackward0>)